<figure>
  <IMG src="figures/logo-esi-sba.png" WIDTH=300 height="100" ALIGN="right">
</figure>

# Software Engineering for Data Science 
## Final Exam: Fall 2025-Winter 2026
## Exam Duration: 02:30 Hours

*By Dr. Belkacem KHALDI (b.khaldi@esi-sba.dz)*

## **Revised Exam Instruction:**

During this exam, you may consult only your lecture notes, lab materials, and personal documentation. The use of large language models (LLMs), whether offline or online, is strictly prohibited. Any violation of this policy constitutes academic misconduct and will result in sanctions, including a grade of zero for the affected exercise.

Adherence to these rules is required to ensure fairness and integrity in the assessment process. Direct any questions or concerns to the exam proctor or instructor.


## **Note: Exam Requirements & Continuous Evaluation Grade**

This exam consists of two mandatory challenges. Both challenges must be completed to a satisfactory standard to pass the exam.

However, for your **Final Continuous Evaluation Grade**, only **one** of the two challenges will be formally assessed and contribute to your grade. You must choose which challenge you wish to be considered for this evaluation.

Please review the options below and decide which challenge you would like to submit for your continuous evaluation grade.

---

### **Continuous Evaluation Challenge Options (Choose One)**

**Option 1: Challenge 1: Data Wrangling, Scraping, and Cleaning for Strategic HRST Planning**
*   **Focus:** This option assesses your skills in advanced data ingestion, transformation, and database manipulation. This may include ingesting data from quering a database schema (e.g., a snowflake schema), and integrating data from different resources like  web scraping, csv, and json files.

**Option 2: Challenge 2: Interactive Web Application Development**
*   **Focus:** This option assesses your ability to apply the cleaned data to build a user-facing analytical tool. You will develop an interactive web application using either **Gradio** or **Streamlit** to visualize key population metrics, as detailed in the "Developing a Web Application for the African Union" challenge. **Bonus points will be awarded for applications built with Gradio.**

---

**Next Steps:**
1.  **For the Exam:** Complete both challenges to the best of your ability.
2.  **For Your Final Continous Evaluation Grade:** Decide which of the two challenges (Option 1 or Option 2) you want to be formally graded.
3.  **Submission:** Clearly state your choice (e.g., "I choose Option 2 for my continuous evaluation grade.") in the comments of your final submission.

In [ ]:
# I choose Option 1 for my continuous evaluation grade.


## **Challenge 1: Data Wrangling, Scraping, and Cleaning for Strategic HRST Planning**

### **Objective:**
You have just taken a role as a Data Scientist at the Department of Human Resources, Science and Technology (HRST) within the African Union Organization. Your task is to gather and clean data related to population statistics across the continent. This data is crucial for informing strategic planning in human capital development and potential technological infrastructure deployment. You will need to integrate data from various sources, including web scraping for specific subregions, querying a central database, and processing CSV and json files. Subsequently, you will perform necessary data cleaning and analysis to ensure the quality and usability of the data for policy-making.

### **Key Points to Note:**
- **Comprehensive Data Integration:** 
   You should integrate data from CSV and json files, a SQLite database, and web scraping into one unified DataFrame.

- **Data Cleaning:**
    Emphasis should be placed on cleaning the data to ensure consistency, accuracy, and usability for high-level strategic analysis.

- **Handling Failures:** 
   You are encouraged to perform data integration to the best of your ability, even if you fail with some of the requirements of the followining parts. The final cleaned CSV file should include data from the successfully ingested sources, while clearly documenting any issues faced with the other sources in the notebook. You should save the final successful cleaned CSV file as `resources/<your_name>_HRST_Africa_Population_Cleaned.csv` (e.g.:`bkhaldi_HRST_Africa_Population_Cleaned.csv`), which will be used later for `Challenge 02: Developing a Gradio/Streamlit Web Application for the HRST Department`.

### **Part 1: Data Ingestion from SQLite Database**

**Task:** Connect to the provided SQLite database (`resources/db/population_star.db`) and extract the appropriate data. This database contains historical data for thre sub african regions:`Eastern Africa`,`Western Africa`, and `Middle Africa`  that will serve as a baseline for the HRST analysis.

**Requirements:**
*   The database is a dementinal database with a Star Schema and contains the following tables `FactPopulation`, `DimGeography`, and `DimTime`.
*   The resulting DataFrame should be named `df_db`.

<figure>
  <IMG src="figures/sqllite_db_diagram.png"  ALIGN="center">
</figure>

#### Data Dictionary: Star Schema Tables

This section provides detailed explanations for each column in the star schema database tables to ensure a clear understanding of the data structure.

##### **FactPopulation**

The central fact table containing all numerical population metrics. The grain is one row per country per year.

| Column Name | Description |
|---|---|
| `FactKey` | Surrogate primary key for the fact table. |
| `TimeKey` | Foreign key linking to the `DimTime` table. |
| `GeographyKey` | Foreign key linking to the `DimGeography` table. |
| `Population` | Total population for the country in a given year. |
| `YearlyChange` | Absolute change in total population over the preceding five-year period (or last year for 2025 data). |
| `YearlyPercentChange` | Percentage change in total population over the preceding five-year period (or last year for 2025 data). |
| `Migrants` | The average annual number of immigrants minus the number of emigrants over the preceding five-year period. A negative number indicates more emigrants. |
| `MedianAge` | The age that divides the population into two numerically equal groups. |
| `FertilityRate` | The average number of children an average woman will have during her reproductive period (15 to 49 years old). |
| `Density` | Population density, expressed as people per square kilometer (P/Km²). |
| `UrbanPopPercent` | Urban population as a percentage of the total population. |
| `UrbanPopulation` | Population living in areas classified as urban. |
| `WorldSharePercent` | The country's total population as a percentage of the total World Population. |
| `GlobalRank` | The country's rank in the list of all countries worldwide ranked by population. |
| `WorldPopulation` | The total World Population as of July 1 of the year indicated. |
| `WorldYearlyChange` | Absolute change in total World Population over the preceding five-year period. |
| `WorldYearlyPercentChange` | Percentage change in total World Population over the preceding five-year period. |

##### **DimGeography**

The dimension table containing geographical attributes.

| Column Name | Description |
|---|---|
| `GeographyKey` | Surrogate primary key for the geography dimension. |
| `CountryName` | The name of the country. |
| `Continent` | The continent the country belongs to. |
| `Subregion` | The subregion within the continent. |

##### **DimTime**

The dimension table containing time-related attributes.

| Column Name | Description |
|---|---|
| `TimeKey` | Surrogate primary key for the time dimension. |
| `Year` | The calendar year. |
| `Decade` | The decade (e.g., 2020). |



In [ ]:
import sqlite3
import pandas as pd

db_path = 'resources/db/population_star.db'
conn = sqlite3.connect(db_path)

query = """
SELECT 
    dt.Year,
    dg.CountryName,
    dg.Continent,
    dg.Subregion,
    fp.Population,
    fp.YearlyChange,
    fp.YearlyPercentChange,
    fp.Migrants,
    fp.MedianAge,
    fp.FertilityRate,
    fp.Density,
    fp.UrbanPopPercent,
    fp.UrbanPopulation,
    fp.WorldSharePercent,
    fp.GlobalRank,
    fp.WorldPopulation,
    fp.WorldYearlyChange,
    fp.WorldYearlyPercentChange
FROM FactPopulation fp
JOIN DimGeography dg ON fp.GeographyKey = dg.GeographyKey
JOIN DimTime dt ON fp.TimeKey = dt.TimeKey
"""

df_db = pd.read_sql_query(query, conn)
conn.close()

print(f"Database records loaded: {len(df_db)}")
print(df_db.head())

### **Part 2: Data Ingestion from CSV and json Files**

**Task:** Read and combine data from multiple CSV and json files located in the `resources/Southern_Africa/` directory. These files were manually compiled by field officers in the **Southern_Africa** subregion and need to be integrated into our central dataset.

**Requirements:**
*   Parse each CSV and json file into a DataFrame.
*   Combine appropriate individual DataFrames into a single appropriate DataFrame named `df_csv` for csv dataframes and `df_json` for json dataframes.


In [ ]:
import os
import json

csv_files = [
    'resources/Southern_Africa/csv/southern_africa_eswatini.csv',
    'resources/Southern_Africa/csv/southern_africa_lesotho.csv',
    'resources/Southern_Africa/csv/southern_africa_namibia.csv',
    'resources/Southern_Africa/csv/southern_africa_south_africa.csv'
]

csv_dataframes = []
for file in csv_files:
    df = pd.read_csv(file)
    csv_dataframes.append(df)

df_csv = pd.concat(csv_dataframes, ignore_index=True)

json_file = 'resources/Southern_Africa/json/southern_africa_botswana.json'
with open(json_file, 'r') as f:
    json_data = json.load(f)

df_json = pd.DataFrame(json_data)

print(f"CSV records loaded: {len(df_csv)}")
print(f"JSON records loaded: {len(df_json)}")
print(df_csv.head())
print(df_json.head())

### **Part 3: Data Ingestion from Web Scraping**

**Task:** Scrape the latest available detailed population data for every country within the **Northern Africa** subregion from the Worldometers local website (available in `resources\wordometer` folder). This requires a two-step automated scraping process to gather comprehensive historical data for this key region.

**Note: Offline Protocol (FILE)**

**For offline scraping**, where the source is a local HTML file, the protocol must be changed to `file://` rather than the `http://` protocol. This protocol instructs the web browser to bypass the internet entirely. Instead, it directly accesses the local file system of the computer on which the script is running. The browser then opens and renders the specified file.

- **For a Windows User:** The script will construct a full path like `C:\Users\YourName\your_project_folder\data\target.htm`. The final URL the browser opens will be `file:///C:/Users/YourName/your_project_folder/data/target.htm`.
- **For a Linux/macOS User:** The script will construct a full path like `/home/yourname/your_project_folder/data/target.htm`. The final URL will be `file:///home/yourname/your_project_folder/data/target.htm`.

A working guide example using selenium to access an offline web page:

``` python
from selenium import webdriver
import os

# Initialize the WebDriver (example for Chrome)
driver = webdriver.Chrome()

# Define the relative path to your HTML file
# Use forward slashes (or double backslashes on Windows)
relative_path = "Users\\YourName\\your_project_folder\\data\\target.htm"

# Get the current working directory and join with the relative path
# os.path.join handles the correct slash for the OS
abs_file_path = os.path.join(os.getcwd(), relative_path)

# Add the 'file:///' protocol prefix (three slashes are for the protocol, 
# and the path starts with one more if it's the root on Linux/macOS)
url = "file:///" + abs_file_path

# On Windows, you might need to adjust for the drive letter prefix
# if using an older Selenium version, but the 'file:///' prefix 
# with the output of os.path.join generally works across platforms.

# Navigate to the local file
driver.get(url)

# Proceed with your web scraping actions
print(f"Opened URL: {driver.current_url}")
# ... your scraping code here ...

# Close the browser
driver.quit()

```



**Requirements:**
1.  **Step 1: Extract Country Links:** First, visit the Northern Africa subregional page (`resources\wordometer\northern-africa-population.htm`) and scrape the list of all countries. For each country, extract its name and navigate to its corresponding link to scrap its detailed population page (see Algeria page as example: `resources\wordometer\northan-africa-countries\algeria-population.htm`).
2.  **Step 2: Iterate and Scrape Details:** Next, write a loop that iterates through the list of country links extracted in Step 1. For each link, navigate to the country's detailed page and scrape the full historical population data table.
3.  **Data Combination:** Combine the scraped data from all countries into a single, unified DataFrame named `df_web`.
4.  **Data Cleaning:** After scraping, ensure the DataFrame is clean while adding the required columns such as `continent` and `subregion`, and standardize column names to match the database DataFrame


<figure>
  <IMG src="figures/north_africa_subregion_webscraping.png"  ALIGN="center">
</figure>

<figure>
  <IMG src="figures/algeria_example_detailed_info_to_scrap.png"  ALIGN="center">
</figure>



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()

main_page_path = os.path.abspath('resources/wordometer/northern-africa-population.htm')
main_url = 'file:///' + main_page_path

driver.get(main_url)
time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')
country_table = soup.find('table')
country_links = []

if country_table:
    rows = country_table.find_all('tr')[1:]
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 0:
            link = cells[0].find('a')
            if link and link.get('href'):
                country_name = link.text.strip()
                href = link.get('href')
                country_links.append({'name': country_name, 'href': href})

all_country_data = []

for country in country_links:
    country_page_path = os.path.abspath(f'resources/wordometer/northan-africa-countries/{country["href"]}')
    country_url = 'file:///' + country_page_path
    
    driver.get(country_url)
    time.sleep(2)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tables = soup.find_all('table')
    
    for table in tables:
        headers = [th.text.strip() for th in table.find_all('th')]
        if 'Year' in headers and 'Population' in headers:
            rows = table.find_all('tr')[1:]
            for row in rows:
                cells = row.find_all('td')
                if len(cells) >= len(headers):
                    row_data = {headers[i]: cells[i].text.strip() for i in range(len(headers))}
                    row_data['CountryName'] = country['name']
                    all_country_data.append(row_data)
            break

driver.quit()

df_web = pd.DataFrame(all_country_data)
df_web['Continent'] = 'Africa'
df_web['Subregion'] = 'Northern Africa'

print(f"Web scraped records: {len(df_web)}")
print(df_web.head())

### **Part 4: Data Integration and Cleaning**

**Task:** Now, your goal is to combine all the DataFrames that you have ingested from different sources into a single, clean, and unified DataFrame named `final_africa_df`.

This is a critical step in any data project. You must assess the data from each source, identify inconsistencies, and apply the necessary transformations to create a reliable and coherent dataset.

**Documentation is Key:**
You are expected to document your entire process. In the code cell below and in subsequent Markdown cells, you must clearly explain:
*   **What problems** you identified in the data (e.g., mismatched columns, incorrect data types, missing values, duplicates).
*   **What steps** you took to clean and preprocess the data.
*   **Why** you chose to perform each step. Your justification is as important as the code itself.

---
**Final Output:**
*   Save your final, cleaned DataFrame to the path `resources/<your_name>_HRST_Africa_Population_Cleaned.csv`.
*   This file will be the official dataset used in the next challenge for developing the Gradio web application.

**Validation:**
*   Display the `head()` of your final DataFrame.
*   Use the `.info()` method to show a summary of the columns, data types, and non-null values.
*   Print the total number of unique countries and the total number of rows in your final dataset.


In [ ]:
df_combined = pd.concat([df_csv, df_json], ignore_index=True)

column_mapping = {
    'Yearly % Change': 'YearlyPercentChange',
    'Yearly Change': 'YearlyChange',
    'Migrants (net)': 'Migrants',
    'Median Age': 'MedianAge',
    'Fertility Rate': 'FertilityRate',
    'Density (P/Km²)': 'Density',
    'Urban Pop %': 'UrbanPopPercent',
    'Urban Population': 'UrbanPopulation',
    "Country's Share of World Pop": 'WorldSharePercent',
    'World Population': 'WorldPopulation',
    'Algeria Global Rank': 'GlobalRank'
}

df_web_cleaned = df_web.rename(columns=column_mapping)

def clean_numeric(value):
    if pd.isna(value):
        return value
    if isinstance(value, str):
        value = value.replace(',', '').replace('%', '').strip()
        try:
            return float(value)
        except:
            return None
    return value

numeric_columns = ['Population', 'YearlyChange', 'YearlyPercentChange', 'Migrants', 
                   'MedianAge', 'FertilityRate', 'Density', 'UrbanPopPercent', 
                   'UrbanPopulation', 'WorldSharePercent', 'GlobalRank', 
                   'WorldPopulation', 'WorldYearlyChange', 'WorldYearlyPercentChange']

for col in numeric_columns:
    if col in df_web_cleaned.columns:
        df_web_cleaned[col] = df_web_cleaned[col].apply(clean_numeric)
    if col in df_combined.columns:
        df_combined[col] = df_combined[col].apply(clean_numeric)

if 'Year' in df_web_cleaned.columns:
    df_web_cleaned['Year'] = pd.to_numeric(df_web_cleaned['Year'], errors='coerce')
if 'Year' in df_combined.columns:
    df_combined['Year'] = pd.to_numeric(df_combined['Year'], errors='coerce')

final_africa_df = pd.concat([df_db, df_combined, df_web_cleaned], ignore_index=True)

final_africa_df = final_africa_df.drop_duplicates(subset=['Year', 'CountryName'], keep='first')
final_africa_df = final_africa_df.sort_values(['CountryName', 'Year']).reset_index(drop=True)

output_path = 'resources/student_HRST_Africa_Population_Cleaned.csv'
final_africa_df.to_csv(output_path, index=False)

print(f"Final dataset saved to {output_path}")
print(f"\nDataset shape: {final_africa_df.shape}")
print(f"Unique countries: {final_africa_df['CountryName'].nunique()}")
print(f"Year range: {final_africa_df['Year'].min()} - {final_africa_df['Year'].max()}")
print("\nDataset info:")
print(final_africa_df.info())
print("\nFirst few rows:")
print(final_africa_df.head())

## **Challenge 02: Developing a Web Application for the African Union**


### **Objective:**
Using your cleaned CSV file `resources/<your_name>_HRST_Africa_Population_Cleaned.csv`, you are tasked with creating an interactive web application. You may choose to build this application using either **Gradio** or **Streamlit**. This dashboard will serve as a tool for policymakers at the African Union's Department of Human Resources, Science and Technology (HRST) to explore and visualize key population metrics across the continent.

### **Key Points to Note:**
- **Platform Choice:** You may use either the `gradio` or `streamlit` library to build the web interface. **Bonus points will be awarded to students who successfully build their application using Gradio.**
- **Interactivity:** Leverage your chosen framework's features (e.g., Gradio's event handlers or Streamlit's widget state) to make your visualizations dynamic. A user's selection from a dropdown should trigger an update in a plot or dataframe.
- **Visualization Libraries:** Use libraries like `Plotly` and `Matplotlib` to generate your plots, which can then be rendered within your chosen framework's components.
- **Code Organization:** Structure your code logically, with separate functions for each visualization or data processing task. This will make your application cleaner and easier to manage.

---

### **Required Functionalities:**

Your web application should be organized into separate sections/pages and include the following features:

**1. Data Overview:**
*   Load and display the cleaned dataset (e.g., using `gr.DataFrame` or `st.dataframe`).
*   Provide basic summary statistics of the dataset (e.g., total countries, years covered, etc.) using a markdown component.

**2. Regional Population Analysis:**
*   **Top Populous Countries:** Create a selection widget (e.g., `gr.Dropdown` or `st.selectbox`) to select a subregion. Based on the selection, display a bar chart showing the **top 5 most populous countries** within that subregion for the most recent year in the dataset.
*   **Growth Leaders:** Using the same subregion selection, display another bar chart showing the **top 5 countries with the highest yearly population growth rate** for the most recent year.

**3. Geographical and Temporal Visualization:**
*   **Africa Population Map:** Generate a choropleth map of the African continent showing the population density for a user-selected year. Use a slider widget to allow the user to select the year and dynamically update the map.
*   **Population Trend Race:** Create a bar chart race animation showing the population change of the top 10 most populous African countries over the years. This should be a key feature of your dashboard.
*   **Hierarchical Sunburst Chart:** Generate an interactive Sunburst chart for a user-selected year. The chart should visualize the population hierarchy from **Continent -> Subregion -> Country**, where the size of each segment represents the population of that geographical area.

**4. Demographic Distribution Analysis:**
*   **Metric Distribution:** Use a selection widget to select a demographic metric (e.g., 'Median Age', 'Fertility Rate', 'Urban Pop %'). Display a histogram or KDE plot showing the probability distribution of that selected metric across all African countries for a user-selected year.

**5. Advanced User Interaction:**
*   **File Upload:** Include a file upload component that allows users to upload their own CSV file with the same column structure. The application should dynamically re-process the data and update all visualizations based on the newly uploaded file.
*   **Multi-page Interface:** Organize all these functionalities into a clean, multi-page interface using the tabbing or navigation features of your chosen framework (e.g., `gr.Tabs()` or `st.tabs()`).

---



### **End of Challenge**

Please save your final application script (e.g., `<your_name>_hrst_population_dashboard.py`) and make sure your application is fully functional.

In [ ]:
import gradio as gr
import plotly.express as px
import plotly.graph_objects as go

df = pd.read_csv('resources/student_HRST_Africa_Population_Cleaned.csv')

def get_top_populous(subregion, df_input):
    recent_year = df_input['Year'].max()
    filtered = df_input[(df_input['Subregion'] == subregion) & (df_input['Year'] == recent_year)]
    top5 = filtered.nlargest(5, 'Population')
    fig = px.bar(top5, x='CountryName', y='Population', title=f'Top 5 Populous Countries in {subregion}')
    return fig

def get_top_growth(subregion, df_input):
    recent_year = df_input['Year'].max()
    filtered = df_input[(df_input['Subregion'] == subregion) & (df_input['Year'] == recent_year)]
    top5 = filtered.nlargest(5, 'YearlyPercentChange')
    fig = px.bar(top5, x='CountryName', y='YearlyPercentChange', title=f'Top 5 Growth Rates in {subregion}')
    return fig

def create_map(year, df_input):
    filtered = df_input[df_input['Year'] == year]
    fig = px.choropleth(filtered, locations='CountryName', locationmode='country names',
                        color='Density', hover_name='CountryName',
                        title=f'Population Density in Africa ({year})')
    return fig

def create_sunburst(year, df_input):
    filtered = df_input[df_input['Year'] == year]
    fig = px.sunburst(filtered, path=['Continent', 'Subregion', 'CountryName'],
                      values='Population', title=f'Population Hierarchy ({year})')
    return fig

def create_distribution(metric, year, df_input):
    filtered = df_input[df_input['Year'] == year]
    fig = px.histogram(filtered, x=metric, title=f'{metric} Distribution ({year})')
    return fig

def process_upload(file):
    if file is not None:
        return pd.read_csv(file.name)
    return df

with gr.Blocks() as demo:
    gr.Markdown("# HRST Population Dashboard")
    
    with gr.Tab("Data Overview"):
        df_display = gr.DataFrame(df.head(100))
        gr.Markdown(f"Total Countries: {df['CountryName'].nunique()}, Years: {df['Year'].min()}-{df['Year'].max()}")
    
    with gr.Tab("Regional Analysis"):
        subregion = gr.Dropdown(choices=df['Subregion'].unique().tolist(), label="Select Subregion")
        plot1 = gr.Plot()
        plot2 = gr.Plot()
        subregion.change(get_top_populous, inputs=[subregion, gr.State(df)], outputs=plot1)
        subregion.change(get_top_growth, inputs=[subregion, gr.State(df)], outputs=plot2)
    
    with gr.Tab("Geographic Visualization"):
        year_slider = gr.Slider(minimum=int(df['Year'].min()), maximum=int(df['Year'].max()), 
                                step=1, label="Select Year")
        map_plot = gr.Plot()
        year_slider.change(create_map, inputs=[year_slider, gr.State(df)], outputs=map_plot)
    
    with gr.Tab("Demographic Distribution"):
        metric_choice = gr.Dropdown(choices=['MedianAge', 'FertilityRate', 'UrbanPopPercent'], 
                                   label="Select Metric")
        year_choice = gr.Slider(minimum=int(df['Year'].min()), maximum=int(df['Year'].max()), 
                               step=1, label="Select Year")
        dist_plot = gr.Plot()
        metric_choice.change(create_distribution, inputs=[metric_choice, year_choice, gr.State(df)], 
                           outputs=dist_plot)
    
    with gr.Tab("Upload Data"):
        file_upload = gr.File(label="Upload CSV")
        upload_status = gr.Textbox(label="Status")

demo.launch()

## **Submission Instructions:**

- **Compression & Naming:** Compress all your solution into a single ZIP folder and name it using the following format:
    `<full_name>_G<#n>_SEDS_EXAM_F25W26.zip`. Replace `<full_name>` with your full name and `<#n>` with your group number (e.g., G3)